In [5]:
# Sample data
data_dict = {
    'Governance': 75,
    'Data Management': 90,
    'Model Development': 30,
    'Deployment': 70,
    'Monitoring and Evaluation': 20,
    'Continuous Improvement': 95
}
data = list(data_dict.items())

# plotly stellar plot

In [6]:
import warnings
warnings.filterwarnings('ignore')

### Helper functions

In [7]:
from itertools import chain, zip_longest
from math import ceil, pi


# We will also need some helper functions, namely a function to round up to the nearest 10
# (round_up()) and a function to join two sequences (even_odd_merge()).
# In the latter, the values of the first sequence (a list or a tuple, basically)
# will fill the even positions and the values of the second the odd ones.
def round_up(value):
    """

    >>> round_up(25)

    30

    """
    return int(ceil(value / 10.0)) * 10


def even_odd_merge(even, odd, filter_none=True):
    """

    >>> list(even_odd_merge([1,3], [2,4]))

    [1, 2, 3, 4]

    """
    if filter_none:
        return filter(None.__ne__, chain.from_iterable(zip_longest(even, odd)))

    return chain.from_iterable(zip_longest(even, odd))

# That said, to plot data on a stellar chart, we need to apply some transformations,
# as well as calculate some auxiliary values. So, let's start by creating a function 
# (prepare_angles()) to calculate the angle of each axis on the chart (N corresponds 
# to the number of variables to be plotted).
def prepare_angles(N):
    angles = [n / N * 2 * pi for n in range(N)]

    # Repeat the first angle to close the circle

    angles += angles[:1]

    return angles

# Next, we need a function (prepare_data()) responsible for adjusting the original 
# data (data) and separating it into several easy-to-use objects.
def prepare_data(data):
    labels = [d[0] for d in data]  # Variable names

    values = [d[1] for d in data]

    # Repeat the first value to close the circle

    values += values[:1]

    N = len(labels)
    angles = prepare_angles(N)

    return labels, values, angles, N


# Lastly, for this specific type of chart, we require a function (prepare_stellar_aux_data())
# that, from the previously calculated angles, prepares two lists of auxiliary values:
# a list of intermediate angles for each pair of angles (stellar_angles) and a list of small
# constant values (stellar_values), which will act as the values of the variables
# to be plotted in order to achieve the star-like shape intended for the stellar chart.
def prepare_stellar_aux_data(angles, ymax, N):
    angle_midpoint = pi / N

    stellar_angles = [angle + angle_midpoint for angle in angles[:-1]]
    stellar_values = [0.05 * ymax] * N

    return stellar_angles, stellar_values

# At this point, we already have all the necessary ingredients for the stellar chart,
# so let's move on to the Matplotlib side of this tutorial. In terms of aesthetics,
# we can rely on a function (draw_peripherals()) designed for this specific purpose 
# (feel free to customize it!).
def draw_peripherals(ax, labels, angles, ymax, outer_color, inner_color, label_color):
    # X-axis

    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels, color=label_color, size=8)

    # Y-axis

    ax.set_yticks(range(10, ymax, 10))
    ax.set_yticklabels(range(10, ymax, 10), color=inner_color, size=7)
    ax.set_ylim(0, ymax)
    ax.set_rlabel_position(0)

    # Both axes

    ax.set_axisbelow(True)

    # Boundary line

    ax.spines["polar"].set_color(outer_color)

    # Grid lines

    ax.xaxis.grid(True, color=inner_color, linestyle="-")
    ax.yaxis.grid(True, color=inner_color, linestyle="-")



In [8]:
import plotly.graph_objects as go

def draw_stellar_plotly(
    labels,
    values,
    angles,
    N,
    shape_color="tab:blue",
    outer_color="slategrey",
    inner_color="lightgrey",
    label_color="black",
    lollipop_background=True,
    lollipop_cmap="jet",
    display_values=False
):
    # Limit the Y-axis according to the data to be plotted
    ymax = round_up(max(values))

    # Get the lists of angles and variable values
    stellar_angles, stellar_values = prepare_stellar_aux_data(angles, ymax, N)
    all_angles = list(even_odd_merge(angles, stellar_angles))
    all_values = list(even_odd_merge(values, stellar_values))
    
    # Create a polar chart
    fig = go.Figure(data=go.Scatterpolar(
        r=all_values,
        theta=all_angles,
        fill='toself',
        line=dict(color=shape_color, width=1),
        name='Stellar Chart'
    ))
    
    # Add a small hole in the center of the chart
    fig.add_trace(go.Scatterpolar(
        r=[0],
        theta=[0],
        mode='markers',
        marker=dict(size=3, color='white')
    ))
    
    # Add lollipops if lollipop_background is True
    if lollipop_background:
        if lollipop_cmap:
            fig.add_trace(go.Scatterpolar(
                r=values,
                theta=angles,
                mode='markers',
                marker=dict(size=20, color=values, colorscale=lollipop_cmap, showscale=True, cmin=min(values), cmax=max(values)),
                name='Data Points'
            ))
        else:
            fig.add_trace(go.Scatterpolar(
                r=values,
                theta=angles,
                mode='markers',
                marker=dict(size=20, color='blue', line=dict(color='red', width=1)),
                name='Data Points'
            ))
    # Add labels if display_values is True
    if display_values:
        for angle, value in zip(angles, values):
            fig.add_trace(go.Scatterpolar(
                r=[value],
                theta=[angle],
                mode='text',
                text=str(int(value)),
                textfont=dict(color='black', size=8),
                name='Data Labels'
            ))
    
    # Set the layout of the chart
    layout = go.Layout(
        polar=dict(
            radialaxis=dict(visible=True, range=[0, ymax]),
            angularaxis=dict(tickangle=-90)
        ),
        showlegend=False,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'
    )

    # Create the figure and plot
    fig = go.Figure(data=data, layout=layout)
    fig.show()

In [9]:
labels, values, angles, N = prepare_data(data)

draw_stellar_plotly(
    shape_color="aliceblue",
    outer_color="slategrey",
    inner_color="lightgrey",
    lollipop_background=True,
    lollipop_cmap='rdylgn', #
    display_values=True,
    *prepare_data(data)
)

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [('Governance', 75), ('Data Management', 90), ('Model Development', 30), ('Deployment', 70), ('Monitoring and Evaluation', 20), ('Continuous Improvement', 95)]

    The 'data' property is a tuple of trace instances
    that may be specified as:
      - A list or tuple of trace instances
        (e.g. [Scatter(...), Bar(...)])
      - A single trace instance
        (e.g. Scatter(...), Bar(...), etc.)
      - A list or tuple of dicts of string/value properties where:
        - The 'type' property specifies the trace type
            One of: ['bar', 'barpolar', 'box', 'candlestick',
                     'carpet', 'choropleth', 'choroplethmapbox',
                     'cone', 'contour', 'contourcarpet',
                     'densitymapbox', 'funnel', 'funnelarea',
                     'heatmap', 'heatmapgl', 'histogram',
                     'histogram2d', 'histogram2dcontour', 'icic